In [12]:
import os
import pinecone
from os.path import join, dirname
from dotenv import load_dotenv

# load environment
env_filepath = os.path.join(dirname('__file__'), '.env')

load_dotenv(env_filepath)

pinecone_api_key = os.getenv("PINECONE_API_KEY")
env_name = "gcp-starter"

pinecone.init(api_key=pinecone_api_key, environment=env_name)

In [13]:
# Check pinecone-client compatibility
import pinecone.info
import console

version_info = pinecone.info.version()
server_version = ".".join(version_info.server.split(".")[:2])
print(f"Pinecone server version: {server_version}")
client_version = ".".join(version_info.client.split(".")[:2])
print(f"Pinecone client version: {client_version}")

assert client_version == server_version, "Pinecone server and client versions do not match. PLease  update the Pinecone client."

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Pinecone server version: 2.0
Pinecone client version: 2.2


AssertionError: Pinecone server and client versions do not match. PLease  update the Pinecone client.